In [5]:
import json
import datetime
import pysrt
from os.path import join
from math import floor, ceil

fps = 23.976
filename = "results.json"
subs_file = "Rogue.One.2016.BRRip.XViD.AC3-ETRG.srt"
#estsub_file = ""
estsub_file = "Rogue.One.2016.1080p.BluRay.x264-SPARKS.srt"
outfile = "capt_results.json"
cntr = []

with open(filename) as f:
    data = json.load(f)
    imdata = {int(frame["img_name"][5:10]):frame for frame in data['results']}
    
    keyw_subs = pysrt.open(join("keyw_subs", subs_file))
    orig_subs = pysrt.open(join("subtitles", subs_file))
    if estsub_file:
        est_subs = pysrt.open(join("est_subs", estsub_file))
        subeix = 0
        esub_start = floor(est_subs[subeix].start.ordinal/1000*fps)
        esub_end = ceil(est_subs[subeix].end.ordinal/1000*fps)
    
    datacounter = 0
    
    subix = 0
    sub_start = floor(keyw_subs[subix].start.ordinal/1000*fps)
    sub_end = ceil(keyw_subs[subix].end.ordinal/1000*fps)
                                                  
    for fr_time, frame in sorted(imdata.items(), key=lambda x: x[0]):
        while fr_time >= sub_end:
            subix += 1
            sub_start = floor(keyw_subs[subix].start.ordinal/1000*fps)
            sub_end = ceil(keyw_subs[subix].end.ordinal/1000*fps)
            #print(fr_time, sub_end)
        if estsub_file:
            while fr_time >= esub_end:
                subeix += 1
                esub_start = floor(est_subs[subeix].start.ordinal/1000*fps)
                esub_end = ceil(est_subs[subeix].end.ordinal/1000*fps)
        if fr_time >= sub_start and fr_time <= sub_end:
            frame["subs"] = orig_subs[subix].text
            frame["keywords"] = keyw_subs[subix].text
            cntr.append(fr_time)
        if estsub_file and fr_time >= esub_start and fr_time <= esub_end:
            frame["estsub"] = est_subs[subeix].text
    data["results"] = list(imdata.values())
    with open(outfile, "w") as g:
        json.dump(data, g)
                
print("Nr of frames: %d and nr of frams with captions: %d." % (len(imdata), len(cntr)))

Nr of frames: 1147 and nr of frams with captions: 491.


In [ ]:
# Old version 
"""
import json
import datetime
import pysrt
from os.path import join

def frame_to_time(framename, fps=24):
    # framename is something like this: scene00001.png
    # https://stackoverflow.com/questions/775049/python-time-seconds-to-hms
    frame = int(framename[5:10])
    seconds = round(frame / fps,3)
    temp = datetime.datetime(1900, 1, 1, 0, 0) + datetime.timedelta(seconds=seconds)
    return temp.time()


filename = "results.json"
subs_file = "Rogue.One.2016.BRRip.XViD.AC3-ETRG.srt"
estsub_file = "Rogue.One.2016.1080p.BluRay.x264-SPARKS.srt"
outfile = "capt_results.json"
cntr = 0

with open(filename) as f:
    data = json.load(f)
    imdata = data['results']
    keyw_subs = pysrt.open(join("keyw_subs", subs_file))
    orig_subs = pysrt.open(join("subtitles", subs_file))
    if estsub_file:
        est_subs = pysrt.open(join("est_subs", estsub_file))
        subeix = 0
        esub_start = datetime.datetime.strptime(str(est_subs[subeix].start), "%H:%M:%S,%f").time()
        esub_end = datetime.datetime.strptime(str(est_subs[subeix].end), "%H:%M:%S,%f").time()
    
    datacounter = 0
    
    subix = 0
    sub_start = datetime.datetime.strptime(str(keyw_subs[subix].start), "%H:%M:%S,%f").time()
    sub_end = datetime.datetime.strptime(str(keyw_subs[subix].end), "%H:%M:%S,%f").time()
                                                  
    for fix, frame in enumerate(imdata):
        timep = frame_to_time(frame["img_name"], fps=23.976)
        while timep >= sub_end:
            subix += 1
            sub_start = datetime.datetime.strptime(str(keyw_subs[subix].start), "%H:%M:%S,%f").time()
            sub_end = datetime.datetime.strptime(str(keyw_subs[subix].end), "%H:%M:%S,%f").time()
        if estsub_file:
            while timep >= esub_end:
                subeix += 1
                esub_start = datetime.datetime.strptime(str(keyw_subs[subeix].start), "%H:%M:%S,%f").time()
                esub_end = datetime.datetime.strptime(str(keyw_subs[subeix].end), "%H:%M:%S,%f").time()
        if timep >= sub_start and timep <= sub_end:
            frame["subs"] = orig_subs[subix].text
            frame["keywords"] = keyw_subs[subix].text
            cntr += 1
        if estsub_file and timep >= esub_start and timep <= esub_end:
            frame["estsub"] = est_subs[subeix].text
    data["results"] = imdata
    with open(outfile, "w") as g:
        json.dump(data, g)
                
print(cntr)
"""